# 1. Hashing task!
In this first task we are implementing hash functions and a structure called Bloom Filter.

In [1]:
import numpy as np
import time
import os
import random as rnd
from collections import defaultdict

In [3]:
# This bloom filter class will have two ins_functions. 
# The first will contain the array representing the bloom filter,
# the second is a list of hash functions which witance attributes:
# Bloom_Filter.array and Bloom_Filter.hashll be used to insert and search elements on the data structure.
class Bloom_Filter:
    
    
    def __init__(self, size, hash_functions):
        self._array = np.empty(size, dtype = bool)
        self._hash_functions = hash_functions
    
   
    def insert(self, element):
        for function in self._hash_functions:
            self._array[function(element)] = True
            
    def check(self, element):
        for function in self._hash_functions:
            if(not self._array[function(element)]):
                return(False)
        return(True)

In [4]:
# #We know the folllowing approximate formula to get a reasonable value of m given an error tolerance p as well as the size n of the elements we are going to insert on the set
#     m = -\frac{n \ln{p}}{(\ln{2})^{2}}

# For the error tolerance $p$ we are going t choose the value $0.01$ so that we'll have only a 1\% rate of false positives.
passwords = open("passwords1.txt", "r")

counter = 0
while(passwords.readline()):
    counter = counter + 1
passwords.close()
print(counter)

100000000


We have found out thar our list consists of 100 milions passwords. 

Knowing this we can finally compute m with the formula given above and get

m =958505838

k = \frac{m}{n}\ln{2}

k = 6.64

So we're going to use seven hash functions.

In [5]:
# We're going to save the minimum as well the maximum possible character in our file
#(characters are ordered by their ASCII code)

passwords = open("passwords1.txt", "r")


minimum = 102
maximum = 102


for _ in range(1000000):
    string = passwords.readline()
    for character in string[:19]: 
        if(ord(character) < minimum):
            minimum = ord(character)
        if(ord(character) > maximum):
            maximum = ord(character)

print(minimum, chr(minimum))
print(maximum, chr(maximum))
passwords.close()

33 !
122 z


We see that every password can contain characters ranging from "!" to "z".

In [8]:
passwords = open("passwords1.txt", "r")


counter = [0] * (122 - 33 + 1)


for _ in range(1000000):
    string = passwords.readline()
    for character in string[:19]:
        counter[ord(character) - 33] += 1

passwords.close()

In [9]:
counter

[226536,
 226375,
 226357,
 226105,
 226044,
 226000,
 226268,
 226404,
 225767,
 225890,
 225885,
 226388,
 226831,
 225541,
 225986,
 226636,
 225616,
 227077,
 226304,
 227385,
 226377,
 225768,
 226336,
 226474,
 226330,
 226024,
 226416,
 226617,
 226811,
 226216,
 226053,
 226097,
 225798,
 226659,
 225852,
 226279,
 226296,
 226135,
 226755,
 226109,
 226002,
 225869,
 226628,
 225940,
 226091,
 226075,
 225593,
 225928,
 225867,
 226701,
 225958,
 226349,
 226193,
 226762,
 225935,
 226347,
 226287,
 226075,
 0,
 0,
 0,
 0,
 0,
 0,
 226032,
 225611,
 226913,
 226309,
 225951,
 226027,
 225492,
 226486,
 225835,
 225963,
 226979,
 226746,
 225956,
 226440,
 226128,
 225894,
 225349,
 225799,
 226374,
 226345,
 225788,
 225929,
 225759,
 225880,
 226971,
 225647]

Every password in the file is a randome string where every character is independently drawn in the set of characters whose ASCII code ranges from 33 to 122, excluding those one whose code ranges from 91 to 96.

In [10]:
def get_base_10(character):
    value = ord(character)
    
    
    if(value < 91):
        return(value - 33)
    else:
        return(value - 39)

 We remember that values ranging from 91 to 96 do not appear

In [11]:
def hash_1(string):
    value  = 0
    for index in range(len(string) - 1, -1, -1):
        value = (84 * value + get_base_10(string[index])) % 958505838
    return(value)

In [12]:
def rotate_string(string, step):
    return(string[step:] + string[:step])

Define our other six functions and since they will be all similar we're going to define an high order function which will take as parameters one hash function (in our case hash_1) and a number $k$ and will return our $k$-th hash function.

In [13]:
def hash_function(first_hash_function, k):
    
   
    return(lambda x : hash_1(rotate_string(x, k - 1)))

When we're going to inizialize our Bloom_Filter class save our hash functions in a list after that we can pass it as a parameter 

In [14]:
hash_functions = [hash_function(hash_1, index + 1) for index in range(7)]

In [15]:
#The function returns the number of strings from the second data set that are possibly contained in the first data set
# and the execution time for finding this number
def task(first_data_set, second_data_set, m, hash_functions):
    
   
    bloom_filter = Bloom_Filter(m, hash_functions)
    
    
    strings = open(first_data_set, "r")
    start = time.time()
    while(True):
        string = strings.readline()
        if(string == ""):
            break
        string = string[:len(string) - 1] 
        bloom_filter.insert(string)
    strings.close()
    
# This can check how many strings from the second data set are probably on the first data set  
# Possible duplicates containing list also created
    strings = open(second_data_set, "r")
    possible_dups = []
    while(True):
        string = strings.readline()
        if(string == ""):
            break
        string = string[:len(string) - 1]
        if(bloom_filter.check(string)):
            possible_dups.append(string)
    end = time.time()
    strings.close()
    
    return((possible_dups, end - start))

In [13]:
if(not os.path.isfile("possible_dups.txt")):
    result = task("passwords1.txt", "passwords2.txt", 958505838, hash_functions)
    f = open("possible_dups.txt", "w")
    f.write(str(result[1]) + "\n")
    for password in result[0]:
        f.write(password + "\n")
    f.close()
else:
    f = open("possible_dups.txt", "r")
    result = [[], 0]
    result[1] = float(f.readline())
    while(True):
        string = f.readline()
        if(string == ""):
            break
        string = string[:len(string) - 1]
        result[0].append(string)
    f.close()

# We print the results
print('Number of hash functions used: ', len(hash_functions))
print('Number of possibly duplicates: ', len(result[0]))
print('Probability of false positives: 0.01')
print('Execution time: ', result[1])

Number of hash functions used:  7
Number of possibly duplicates:  14261334
Probability of false positives: 0.01
Execution time:  6857.633438587189


### BONUS

Assuming that our hash function behaves well (and we hope so based on our discussions in the previous sections) all this process should take an amount of time linear in the size n =100000000 of the first data set, which seems reasonable.

So let's start by creating our array hash_duplicates.

In [15]:
def hash_dictionary(list_of_data, hash_function):
    to_return = defaultdict(list)
    for element in list_of_data:
        to_return[hash_function(element)].append(element)
    return(to_return)
#We now create our dictionary of possibly duplicates

In [16]:
possible_dups_dict = hash_dictionary(result[0], hash_1)

In [22]:
f = open("passwords1.txt", "r")

while(True):
    string = f.readline()
    if(string == ""):
        break
    string = string[:len(string) - 1]
    hash_value = hash_1(string)
    if(string in possible_dups_dict[hash_value]):
        possible_dups_dict[hash_value].remove(string)

f.close()

false_positives = []
for elements in possible_dups_dict.values():
    false_positives.extend(elements)

print("Number of false positives: " + str(len(false_positives)))

Number of false positives: 261334


We can see that the percentage of false positives is approximately 1.8%. This is a bit more than 1% as we initially asked, probably due to the fact that, as we have seen, our hash functions can't be exactly independent.

# 2. Alphabetical Sort

In this exercise we wil focus on sorting characters and strings. 
### Part 1:
For the first part we were asked to implement the Counting Sort algorithm and this is how we did it:

In [1]:
def counting_sort(l):

    range_el = max(l) + 1
    occur = [0] * range_el
    final = [0] * (len(l))

    for i in range(len(l)):
        occur[l[i]] += 1

    for i in range(1, len(occur)):
        occur[i] = occur[i - 1] + occur[i]

    for e in l:
        final[occur[e]-1] = e
        occur[e] -= 1

    return final

### N.B.
* range_el is the range of the elements we are trying to sort, basically it corresponds to the biggest value we have int the list that we need to sort. 
* in the first loop we are counting the occurrences of every element
* in the second loop we are summing each element of the occur vector with his previous one
* in the third loop we are actually creating the ordered list (final) based on the info in the occur list

We used [this video][1] as reference to understand how the counting sort works.


[1]: https://www.youtube.com/watch?v=7zuGmKfUt7s

### Part 2:
Here we wrote an algorithm that uses Counting Sort to sort the letter of the alphabet

In [4]:
def sort_char(char_list):

    to_sort_int = [(ord(x) - 97) for x in char_list]  # turning the characters into int from 0 to 25
    result = counting_sort(to_sort_int)               # running the counting sort on them 
    result = [chr(x + 97) for x in result]            # turning the int back into char
    return result

Let's see how it works:

In [5]:
to_sort = ['p', 'a', 'n', 'd', 'r', 'e', 'q', 'w', 't', 'y', 'u', 'i', 'o', 's', 'f', 'g', 'h', 'j', 'k', 'l', 'z', 'x',
           'c', 'v', 'b', 'm', 'f']

print(sort_char(to_sort))

['a', 'b', 'c', 'd', 'e', 'f', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


#### Time Complexity:
Counting sort is great for sorting positive numbers in a given and realtivly small range because it takes linear time to compute. In fact we don't have nested for loops. Every loop is either looping over len(input) or the range (k) of possible values of the input. In fact the complexity is O(n+k) where n is the length of the input list and k is the range the possible values of the elements in the list.
In our algorithm that works on characters we didn't add complexity because we are only turning into int every char (a loop over n), calling the counting sort and then turning the int back into char (loop over n). So the time complexity remains linear.

### Part 3:
In this part we built an algorithm based on counting sort to sort strings alphabetically. We decided to convert the strings into numbers (turning any char into a two digit number) and sort them basing our algorithm on the radix sort, which recursively calls counting sort to sort numbers first by the most significant digit, then next one and so on until the least significant one. In this algorithm we used some **auxiliary functions:**

In [6]:
# This function is used to prepar the input for the main function

def prep_input(list_strings):

    m = len(max(list_strings, key=len))  # max length of a string in the list
    out = []
    list_strings = [x.lower() for x in list_strings]  # converting to lowercase

    for word in list_strings:            # for every word we prepare the new numerical string
        s = ""
        for letter in word: 
            if letter == " ":            # if the word has spaces in it we represent them with the code '01'
                s += '01'  
            else:
                s += str(ord(letter)-86)  # al the other letters are converted into a number (from 11 to 36)
        if len(word) < m:
            s = s.ljust((len(word) + (m-len(word)))*2, '0')  # if a string is shorter than m we pad it with 
                                                             # zeros at the end to have them of the same length
        out.append(s)

    out = [int(x) for x in out]          # converting all the strings into int

    return out


# This functions is used to convert the numbers back into words after we sorted them

def prettify(ordered_list_int):
    result = []
    for num in ordered_list_int:
        num = str(num)
        stringa = ""                                 # for every number we create a string
        for i in range(1,len(num), 2):               # we have a character every two digits
            if (num[i-1])+str(num[i]) == "00":       # 00 gets ignored beacuse it was just padding
                continue
            elif (num[i-1])+str(num[i]) == "01":     # 01 mean a space inside the string
                stringa += " "
            else:
                stringa += chr(int(str(num[i-1])+str(num[i]))+86)  # all the other numbers are 
                                                                   # turnet back into chars
        result.append(stringa)

    return result

* Then we have the **main functions** of the algorithm:

In [7]:
# This is the same structure of the counting sort above, but with a little change that
# lets us consider only the current decimal position (expressed by digit) to sort the numbers.
# Moreover this time we are modifying the list in place

def counting_sort_snd(l, digit):

    occur = [0] * 10
    final = [0] * (len(l)+1)

    for i in range(len(l)):
        occur[(l[i] // digit) % 10] += 1

    for i in range(1, len(occur)):
        occur[i] = occur[i - 1] + occur[i]

    for e in reversed(l):
        final[occur[(e // digit) % 10] - 1] = e
        occur[(e // digit) % 10] -= 1

    for i in range(len(l)):
        l[i] = final[i]

In [12]:
# This is our version of the radix sort algorithm; it finds out the biggest number in the list to order
# and then calls the counting sort on every decimal position/digit incrementig it every time.

def radix_sort(l):

    range_el = max(l)
    digit = 1

    while range_el / digit > 0:
        counting_sort_snd(l, digit)
        digit *= 10

* and this is the final product:

In [13]:
def my_algorithm(to_sort):

    print("Input: ", to_sort)            # printing input
    prepped_list = prep_input(to_sort)   # preparing input

    radix_sort(prepped_list)             # ordering list
    result = prettify(prepped_list)      # make the result readable

    return result

Let's see it at work:

In [14]:
input_test = ["good", "bad", "building", "oak hill", "zara", "kiss", "kissing", "oak", "crazy", "wow"]

print("Result: ", my_algorithm(input_test))

Input:  ['good', 'bad', 'building', 'oak hill', 'zara', 'kiss', 'kissing', 'oak', 'crazy', 'wow']
Result:  ['bad', 'building', 'crazy', 'good', 'kiss', 'kissing', 'oak', 'oak hill', 'wow', 'zara']


#### Time complexity:
The auxiliary funtions both take almost a quadratic time to be executed beacuse they loop on every word of the list and then on every letter of the word (or every two letters). More precisely they take O(m* n) where m is the number of words and n the maximum lenght of the words. The counting sort is the same so it's still linear. The radix_sort is calling the counting sort for every digit of the number so the total amount of the radix sort is O(m* j) where m is still the number of elements to sort and j is the medium number of digits they have. 

# 3. Find similar wines!
The goal of this exercise is clustering wines by similarity

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from copy import deepcopy

# Reading csv dataset as it does not have header, and separated by names
wine = pd.read_csv("wine.data", names = ["Class", 
                                         "Alcohol",
                                         "Malic acid",
                                         "Ash",
                                         "Alcalinity of ash",
                                         "Magnesium",
                                         "Total phenols",
                                         "Flavanoids",
                                         "Nonflavanoid phenols",
                                         "Proanthocyanins",
                                         "Color intensity",
                                         "Hue",
                                         "OD280/OD315 of diluted wines",
                                         "Proline"])


wine.Class = wine.Class - 1

# K-means from scratch 
X = wine.iloc[:, [12, 1]].values 

data = pd.DataFrame(X)

val1 = data[0].values
val2 = data[1].values

#Euclidean Distance Function
def dist(x, y, ax=1):
    return np.linalg.norm(x - y, axis=ax)

# Number of clusters
k = 3

# Creating random centroids
# random x coordinates of centroids
x_coor = np.random.randint(0, np.max(X)-7, size=k)
# random y coordinates of random centroids
y_coor = np.random.randint(0, np.max(X)-7, size=k)
# Centroids
_centroid = np.array(list(zip(x_coor, y_coor)), dtype=np.float32)
print("Initial Centroids")
print(_centroid)

# Plotting actual values and the random centroids
plt.scatter(val1, val2, c='#050505', s=7)
plt.scatter(x_coor, y_coor, marker='x', s=200, c='g')

# To store the old value of centroids when it updates
my_centroid = np.zeros(_centroid.shape)
clusters = np.zeros(len(X))

# Error function  - Distance between updated and old centroids
error = dist(_centroid, my_centroid, None)

# Loop will run till the error becomes zero
while error != 0:
    # Assigning each value to its closest cluster
    for i in range(len(X)):
        distances = dist(X[i], _centroid)
        cluster = np.argmin(distances)
        clusters[i] = cluster
    # Storing the old centroid values
    C_old = deepcopy(_centroid)
    # Finding the new centroids by taking the average value
    for i in range(k):
        points = [X[j] for j in range(len(X)) if clusters[j] == i]
        _centroid[i] = np.mean(points, axis=0)
    error = dist(_centroid, my_centroid, None)

colors = ['red','blue','green']
fig, ax = plt.subplots()
for i in range(k):
        points = np.array([X[j] for j in range(len(X)) if clusters[j] == i])
        ax.scatter(points[:, 0], points[:, 1], s=7, c=colors[i])
ax.scatter(_centroid[:, 0], _centroid[:, 1], marker='*', s=200, c='#050505')

           
# K means with scikit-learn libraries
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);


# 4. K-means can go wrong!

Since Clustering algorithms are for unsupervised data,chance of getting less optimal result is 
possible.One reason is differently from supervised learning, here we define k(number of clusters) 
as an input,it is not learned from data.In Random Initialization,the initial centroids is randomly
chosen by k-means,which is randomly assigned to some points.In another kind of initialization, 
Forgy Method, the initial points chosen by dataset. Generally, Forgy method is more successful
than Random Initialization. There is also k-means++ which is improved version for handling poor 
clustering. Considering properties of clustering such as number of clusters, cluster overlap, 
balance or unbalance, inilialization is important to get optimal result. In a poor cluster cost 
function(error between actual and predicted value) is high while in good cluster cost function is 
less(because of less error). 